In [ ]:
###
## TO DO

# get_ways_fromNode(node,ways)
# get_node_fromWay(node,way)
# get_time(node_a, node_b, way)

# World.print_world(self)

####

In [2]:
!pip install OSMPythonTools
!pip install overpy

In [3]:
#Not really using this at the moment, but might have to, to do queries between nodes
from OSMPythonTools.api import Api
from OSMPythonTools.overpass import Overpass

api = Api()
queens = api.query('way/12030883') 
overpass = Overpass()

#Bounding box for Campus is (roughly)
#[bbox:44.2224495448,-76.5017059541,44.2298231916,-76.4903552918]


[api] downloading data: way/12030883
The requested data could not be downloaded.  Please check whether your internet connection is working.
Traceback (most recent call last):
  File "/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/urllib/request.py", line 1348, in do_open
    h.request(req.get_method(), req.selector, req.data, headers,
  File "/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/http/client.py", line 1282, in request
    self._send_request(method, url, body, headers, encode_chunked)
  File "/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/http/client.py", line 1328, in _send_request
    self.endheaders(body, encode_chunked=encode_chunked)
  File "/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/http/client.py", line 1277, in endheaders
    self._send_output(message_body, encode_chunked=encode_chunked)
  File "/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/http/client.py", line 1037, in 

Exception: ('The requested data could not be downloaded.  Please check whether your internet connection is working.', URLError(SSLCertVerificationError(1, '[SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: unable to get local issuer certificate (_ssl.c:992)')))

In [87]:
#https://python-overpy.readthedocs.io/en/latest/api.html
import overpy
overpyapi = overpy.Overpass()

queens_campus_nodes = overpyapi.query('[bbox:44.2224495448,-76.5017059541,44.2298231916,-76.4903552918];(node;);out;')
queens_campus_ways = overpyapi.query('[bbox:44.2224495448,-76.5017059541,44.2298231916,-76.4903552918];(way;);out;')
print("All nodes on campus: "+str(len(queens_campus_nodes.nodes)))
print(queens_campus_nodes.nodes[1])
print("All ways on campus: "+str(len(queens_campus_ways.ways)))
print(queens_campus_ways.ways[1])

queens_campus_relations = overpyapi.query('[bbox:44.2224495448,-76.5017059541,44.2298231916,-76.4903552918];(relation;);out;')


All nodes on campus: 10311
<overpy.Node id=27495897 lat=44.2279960 lon=-76.4908268>
All ways on campus: 1569
<overpy.Way id=4642462 nodes=[7684020544, 401544451, 7684020543, 7281272432, 9604229820, 9604229821, 27495910]>


In [142]:
# Sample functions
# Fetch functions for ways connected to node
def get_ways_fromNode(node,ways):
    pass

# Get all nodes in a way
#https://python-overpy.readthedocs.io/en/latest/example.html
def get_node_fromWay(node, way):
    pass
    
# Calulate the time walking a "Way"
def get_time(node_a, node_b, way):
    pass

# maybe get time from

# Search for a tag in an area "query" This could be used for input/output coordinates/location

#[out:json][timeout:250];
#(
#  area[name="Palo Alto"];
#  way(area)["building"];
#  relation(area)["building"];
#);
#out body;
#>;
#out skel qt;
def get_nodes_with_tag(tag, nodes):
    n_list = []
    for n in nodes:
        if tag in vars(n)['tags'].keys():
            n_list.append(n)
    return n_list

# returns name of building
def _node_to_str(node):
    return vars(node)['tags']['name']

# returns name of node (if it exists)
def _str_to_node(string, nodes):
    for n in nodes:
        if string in vars(n)['tags']['name']:
            return n
    return"no node with this name"

# Get random node in set
def random_node(set):
    pass

#Could be used for input/output coordinates/location
def get_closest_node_fromCoordinate(latitude, longitude):
  # figure out how to input coordinate and in what order they are usually shown
  pass
  #Output node
    

In [6]:
# Q-learning / agent -> Figure out what functions you will specifically need from this data 
# Enviroment -> figure out enviroment ~  how to connect the agents actions to the enviroment -> To the api
              #ie: how to break down 

In [147]:
from copy import deepcopy
import numpy as np
class World:
    
    def __init__(self, nodes, ways, start, terminal):
    
        # nodes (list) = the nodes
        # ways (list) = the ways
        # start (node) = the starting node
        # terminal (node) = the goal node

        # policy (dict) = each node a: {each potential goal node: best first way to take to reach this goal}
        # values (dict) = each node a: {each potential goal node: {each way from node a: value}}

        self._nodes = nodes
        self._ways = ways
        self._start = start
        self._terminal = terminal
        
        self._policy = {}
        self._values = {}
        
        self._values_next = {}
        self._values_next = self.create_next_values()
        
        for node in self._nodes:
            self._values[node] = {}
            self._values[node] = dict(zip(self._nodes, [None]*len(self._nodes)))
            self._policy[node] = {}
            self._policy[node] = dict(zip(self._nodes, [None]*len(self._nodes)))
            for goal_node in self._values[node]:
                self._values[node][goal_node] = {}
                self._values[node][goal_node] = dict(zip(get_ways_fromNode(node,self._ways), [None]*len(get_ways_fromNode(node,self._ways))))
                self._policy[node][goal_node] = random.choice(self._values[node][goal_node].keys())
                for possible_way in self._values[node][goal_node]:
                    self._values[node][goal_node][possible_way] = 0
    
    def reset(self):
        for node in self._nodes:
            self._values[node] = {}
            self._values[node] = dict(zip(self._nodes, [None]*len(self._nodes)))
            self._policy[node] = {}
            self._policy[node] = dict(zip(self._nodes, [None]*len(self._nodes)))
            for goal_node in self._values[node]:
                self._values[node][goal_node] = {}
                self._values[node][goal_node] = dict(zip(get_ways_fromNode(node,self._ways), [None]*len(get_ways_fromNode(node,self._ways))))
                self._policy[node][goal_node] = random.choice(self._values[node][goal_node].keys())
                for possible_way in self._values[node][goal_node]:
                    self._values[node][goal_node][possible_way] = 0
        self.create_next_values()
        
    # you get the reward when you take the way
    # a reward of 0 when you reach the terminal state
    # a negative reward when you take an way
    def _get_reward(self, node_a, node_b, way):
        if node_b == self._terminal:
            return 0
        else:
            return -get_time(node_a,node_b,way)
    
    def _get_next_node(self, node, way):
        return get_node_fromWay(node, way)
    
    # returns all nodes in the world
    def get_nodes(self):
        return self._nodes
    
    # returns each way from this node to terminal
    def get_ways(self, node):
        return self._values[node][gw._terminal].keys()
    
    # returns the value of each way from this node to terminal
    def get_values(self, node):
        return self._values[node][gw._terminal]
    
    # values should be a dictionary of {(goal node: (possible ways: value)),}
    def set_values(self, node, values):
        self._values_next[node][gw._terminal] = values
        
    def set_next_values(self):
        self._values = deepcopy(self._values_next)
        
    def create_next_values(self):
        for node in self._nodes:
            self._values_next[node] = {}
            self._values_next[node] = dict(zip(self._nodes, [None]*len(self._nodes)))
            self._policy[node] = {}
            self._policy[node] = dict(zip(self._nodes, [None]*len(self._nodes)))
            for goal_node in self._values[node]:
                self._values_next[node][goal_node] = {}
                self._values_next[node][goal_node] = dict(zip(get_ways_fromNode(node, self._ways), [None]*len(get_ways_fromNode(node, self._ways))))
                self._policy[node][goal_node] = random.choice(self._values_next[node][goal_node].keys())
                for possible_way in self._values[node][goal_node]:
                    self._values_next[node][goal_node][possible_way] = 0
                    
    def print_world(self):
        pass
        

In [146]:
def qlearning(world, alpha, gamma, e, max_eps=500):

    world.reset()
    q = {}

    for node in world.get_nodes():
        q[node] = {}
        if node != world._terminal:
            for way in get_ways_fromNode(node, world._ways):
                q[node][way] = 1
        elif node == world._terminal:
            for way in world.ways:
                q[node][way] = 0
                
    world.create_next_values()
    
    for i in range(0, max_eps):
        n = world._start
        
        while n != world._terminal:   
            rc = np.random.choice(['exploit','explore'], p=[1-e, e])
            if rc == 'exploit':
                v = np.argmax(list(q[n].values()))
                w = list(g[n][v])
            elif rc == "explore":
                w = np.random.choice(list(q[n]))
            n_prime = world._get_next_node(n,w)
            r = world._get_reward(n, n_prime, way)
            q[n][w] += alpha*(r + (gamma * np.max(list(q[n_prime].values())) - q[n][w]))
            values = {}
            for way in get_ways_fromNode(n, world._ways):
                values[way] = q[n][way]
            world.set_values(n, values)
            n = n_prime


        world.set_next_values()
        
    world.create_next_values()

In [153]:
nodes = queens_campus_nodes.nodes
ways = queens_campus_ways.ways

# making a list of campus buildings from ways with the building tag
buildings = []
for w in ways:
    if 'building' in vars(w)['tags'].keys():
        if 'name' in vars(w)['tags'].keys():   
            buildings.append(w)
print("node to string: ", _node_to_str(buildings[5]) )
print("string to node: ", _str_to_node("Ontario Hall", buildings))

start = np.random.choice(nodes)
terminal = np.random.choice(nodes)

# world = World(nodes, ways, start, terminal)
alpha = 0.1
gamma = 0.9
e = 0.1

# qlearning(world, alpha=alpha, gamma=gamma, e=e)
# world.print_world()

node to string:  Ontario Hall
string to node:  <overpy.Way id=5145117 nodes=[35721501, 35721502, 35721503, 6154665917, 6154665916, 35675844, 6154665915, 6154665914, 35675842, 6154665913, 6154665912, 6154665911, 35675841, 35675840, 35675839, 9224895908, 6154665910, 6154665909, 35675837, 6154665908, 35675836, 35675835, 6154665907, 35675834, 35675833, 35675831, 35675829, 9229963425, 9229963426, 6154665906, 35675826, 35675827, 6154665904, 6154665903, 35675825, 35721500, 35721501]>


In [ ]:
class interface():
  pass